In [ ]:
import json
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from evaluate.lane import LaneEval
import random
import os
%matplotlib inline


In [ ]:
def show_samples(n_samples, images, randomize=False):
    """Shows multiple images from path list"""
    for sample_n in range(n_samples):
        if randomize:
            n = random.randint(0, len(images)-1)
        else:
            n = sample_n
        path = os.path.dirname(images[n])
        label = os.path.basename(path)
        #label = images[n][-32:-6]
        fig = plt.figure(figsize=(30,6))
        ax = plt.subplot(np.ceil(n_samples/3), 3, sample_n+1)
        ax.set_title(label)
        img = plt.imread(images[n])
        ax.imshow(img)
        ax.set_yticklabels([])
        ax.set_xticklabels([])
    plt.tight_layout()
    plt.show()


### Get Data And Create small subset for testing

In [ ]:
small_set = False
### READ FILE
data_path = "train_set/"
json_gt = [json.loads(line) for line in open('train_set/label_data_0531.json')]
if small_set:
    subset = np.random.choice(json_gt, size=100, replace = False)
    json_gt=subset

def _get_gt(json_list, index):
    """Reads data from json"""
    gt = json_list[index]
    gt_lanes = gt['lanes']
    y_samples = gt['h_samples']
    raw_file = gt['raw_file']
    return [raw_file, gt_lanes, y_samples]

### Examine Data

In [ ]:
from sys import getsizeof
print("Total data: ",len(json_gt))
example_data = _get_gt(json_gt,2)
print("Path example: ", example_data[0])
print("Lane example: ", example_data[1])
example_img=[]
for i in range(3):
    example_data=_get_gt(json_gt, i)
    example_img.append("train_set/"+ example_data[0])
show_samples(3, example_img)

### Preprocessing

In [ ]:
### Crop, as max index for evaluation is 160, no case
##  in looking above
def roi(img,y):
    """Region of Interest"""
    return(img[y:len(img)])

def draw_lanes(img, gt_lanes, y_samples, thickness=25):
    """raws lines from GT data"""
    gt_lanes_vis = [[(x, y) for (x, y) in zip(lane, y_samples) if x >= 0] for lane in gt_lanes]
    img_vis = np.zeros_like(img)
    if len(gt_lanes_vis)>4:
        gt_lanes_vis = gt_lanes_vis[0:4]
    for lane in gt_lanes_vis:
        lower =[]
        midd =[]
        upper=[]
        for x,y in lane:
            if y < 150+190:
                upper.append([x,y])
            if y >130+190 and y < 150+2*190:
                midd.append([x,y])
            if y > 130 +190*2:
                lower.append([x,y])
        cv2.polylines(img_vis, np.int32([upper]), isClosed=False, color=(0,255,0), thickness= thickness)
        cv2.polylines(img_vis, np.int32([midd]), isClosed=False, color=(0,255,0), thickness= thickness+10)
        cv2.polylines(img_vis, np.int32([lower]), isClosed=False, color=(0,255,0), thickness= thickness+20)
        
    return img_vis

def poly_to_points(poly):
    """Takes a second degree polynomial, returns x_coeffs at y_samples"""
    x=[]
    y=[]
    for y_test in range(160,720,10):
        y.append(y_test)
        pred = y*y*poly[0] + y*poly[1] + poly[2]
        if pred >0 and pred <1280:
            x.append(pred)
        else:
            x.append(-2)
    return x, y

def poly_to_line(img, poly):
    """Draws polynomial to image"""
    line=[]
    for y in range(180,730,10):
        x = y*y*poly[0] + y*poly[1] + poly[2]
        if x>0 and x<1280:
            line.append([np.int32(x), np.int32(y)])
    pts = np.asarray(line, np.int32)
    pts.reshape((-1,1,2))
    output=cv2.polylines(img, [pts], False, (255,0,0), 3)
    return output

def _filter_4lines(poly):
    """Takes polynomial coefficients, if not 4, return only 4
    """
    size=len(poly)
    if size <4:
        for i in range(4-size):
            poly.append(np.asarray([0,0,0]))
    return poly

def _get_coeffs(gt_lanes, y_samples):
    """Returns order 2 polynomials from ground truth points"""
    polys=[]
    gt_lanes_pts = [[(x, y) for (x, y) in zip(lane, y_samples) if x >= 0] for lane in gt_lanes]
    for lane in gt_lanes_pts:
        lane = np.asarray(lane)
        x = lane[:,0]
        y = lane[:,1]
        polys.append(np.polyfit(y,x, 2))
    polys = _filter_4lines(polys)
    return polys
    
def _toBinaryThresh(S_channel, low_limit=0, high_limit=255):
    """Takes saturation channel, outputs binary image"""
    s_binary=np.zeros_like(S_channel)
    s_binary[(S_channel>=low_limit) & (S_channel<=high_limit)]=1
    return s_binary

def generate_GT(data, input_path="train_set/", debbug=False, output_dir="out_img/generate_GT/", save_big_mask= False):
    """Takes data read from GT json, ouputs polynomials, GT_mask and image"""
    path = os.path.dirname(data[0])
    name = os.path.basename(path)
    img = mpimg.imread(input_path +data[0])
    poly_lanes=_get_coeffs(data[1], data[2])
    
    mask=draw_lanes(img, test_data[1], test_data[2],5)
    binary_mask=_toBinaryThresh(mask[:,:,1], 200,255)
    
    hsv_img= cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    v_channel = hsv_img[:,:,2]
    v_channel = _toBinaryThresh(v_channel, 170,255)
    
    h_channel = hsv_img[:,:,0]
    h_channel = _toBinaryThresh(h_channel, 15,25)
    
    gt_patch = cv2.bitwise_or(h_channel, v_channel)
    gt_patch=cv2.bitwise_and(gt_patch, binary_mask)
    
    kernel= np.ones((3,3))
    gt_patch = cv2.morphologyEx(gt_patch,cv2.MORPH_CLOSE, kernel)
    kernel= np.ones((5,7))
    #gt_patch = cv2.dilate(gt_patch, kernel, iterations=1) 
    
    overlay=np.zeros_like(img)
    overlay[:,:,1] = 255*gt_patch
    
    if save_big_mask:
        #mpimg.imsave(GT_binary+"Overlay.jpg",overlay)
        print("save_big_mask not implemented!")
    
    if debbug:
        mpimg.imsave("out_img/" + name + ".jpg",img)
        mpimg.imsave("out_img/"+ name +"GT_lanes_.jpg",mask)
        regression = np.copy(img)
        for poly in poly_lanes:
            regression=poly_to_line(regression, poly)
        mpimg.imsave(output_dir + name +"poly_reversed_GT.jpg",regression)
        mpimg.imsave(output_dir+name+"_v_chan.jpg",v_channel)
        mpimg.imsave(output_dir+name+"_h_chan.jpg",h_channel)
        mpimg.imsave(output_dir+name+"patch.jpg",gt_patch)
        mpimg.imsave(output_dir+name+"Overlay.jpg",overlay)
        result = cv2.addWeighted(overlay,.4,img,.6,0)
        mpimg.imsave(output_dir+name+"Result.jpg",result)
        
    img = cv2.resize(img, None,fx=0.25, fy=0.25)
    overlay = cv2.resize(overlay, None,fx=0.25, fy=0.25)
    
    return name, poly_lanes, overlay, img
    


### Generate GT mask

In [ ]:
#for i in range(len(json_gt)):
for i in range(4):
    test_data = _get_gt(json_gt,i)
    name, poly, mask, img = generate_GT(test_data)
    mpimg.imsave("out_img/" + name + "_mask.jpg",mask)
    mpimg.imsave("out_img/" + name + "_img.jpg",img)

    
    
plt.imshow(cv2.addWeighted(mask, 0.5, img, 0.5, 0))
